# Homework 4
<p> Thomas Dougherty <br>
10-14-2023 </p>

## Problem 1

`What is transfer learning? Look it up, and write how you understood it.`
<p> Transfer learning is the act of training a model for one task (taking a pretrained model) and fine tuning it for another specific task. Transfer learning is a more efficient way of training neural networks because it allows them to use knowledge from the previous task while working with less labeled data. </br>

It's like going from a riding a mountain bike to learning how to ride a motorized dirt bike. All the time spent on a mountain bike taught you balance, cornering technique, and riding over rough terrain. Since those previous skills don't have to br learned again, you can focus on things like throttle control and riding at high speed giving you a headstart on the learning process. </p> </br>

`Explain where the differences are that make the loss plot different between the first two notebooks.`
1. neaclass2 is using a pretrained resnet18 model whereas neaclass1 is using a newly instantiated resnet18 model. This gives it a head start in the training process.
</br>
2. neaclass2 has learning rate scheduler. By slowing incrementally slowing down the learning rate over a period of epochs, the model will find the simple patterns first then "zero in" on finding complex patterns.
</br>

`Finally, run neaclass3.ipynb  and explain the differences with respect to neoclass2.ipynb. At the end of neaclass3.ipynb create a test dataset from  NEUdata_split/Test and  use it to evaluate the accuracy of both its models.`
1. neaclass3 deepcopies the original pre-trained resnet18 model.
2. neaclass3 
<br>
Accuracy for original Resnet18 model came out to 95%, whereas accuracy for the vector model came out to around 25%

In [13]:
import os
from utils import DatasetUtils
import torch
from torch import optim 
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from PIL import Image
from torchvision import transforms
from torchvision.datasets import DatasetFolder
import cv2
import torch.nn as nn
import numpy as np


## Problem 2
### Design a convolutional autoencoder for a dataset of images 3 x 224 x224. Train it on NEU data for 50 epochs.

I'll start with getting the testing & training data and transforms

In [14]:
def load_image(img_path:str):    
        np_img = cv2.imread(img_path)
        return Image.fromarray(np_img)
"""
test_loader = DatasetUtils.create_loader_and_transform(
    root_path='RowanDLclassNEA/NEUdata_split/Test', 
    loader_func=load_image, 
    extensions=('.bmp',), 
    batch_size=32,
    shuffle=False,
    is_test=True
)

train_loader, val_loader, train_set, val_set = DatasetUtils.create_loader_and_transform(
    root_path='NEUdata', 
    loader_func=load_image, 
    extensions=('.bmp',), 
    batch_size=16,
    shuffle=True,
    is_test=False
)

"""
dset = DatasetFolder(root='NEUdata', loader = load_image, extensions = ('.bmp',))

transform = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224),
  transforms.ToTensor(),
  transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225])])


test_xform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225])])

dset = DatasetFolder(root='NEUdata', loader = load_image, extensions = ('.bmp',), transform = transform)
test_data = DatasetFolder(root='RowanDLclassNEA/NEUdata_split/Test/', loader = load_image, extensions=('.bmp'), transform=test_xform)


train_set, val_set = random_split(
                      dset, 
                      [1200, 600])

train_loader = torch.utils.data.DataLoader(
                    train_set, 
                    batch_size=32, 
                    shuffle=True)

val_loader = torch.utils.data.DataLoader(
                    val_set, 
                    batch_size=32, 
                    shuffle=True)


test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size = 32,
            shuffle = False)

print(len(train_loader))
print(len(val_loader))
#print(len(test_loader))
print(train_set)


38
19


### Autoencorder Class

Found that pooling layers were needed for this just to bring the memory usage down to a reasonable level. I experimented with unpooling in the decoder but couldn't get it to work properly with the training loop.

In [15]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        """
            Encoder Layers
        """
        self.enc_conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.enc_pool1 = nn.MaxPool2d(2, stride=2, return_indices=True)

        self.enc_conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.enc_pool2 = nn.MaxPool2d(2, stride=2, return_indices=True)

        self.enc_conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.enc_pool3 = nn.MaxPool2d(2, stride=2, return_indices=True)

        self.enc_conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.enc_pool4 = nn.MaxPool2d(2, stride=2, return_indices=True)

        """
            Decoder Layers
        """
        self.dec_unpool1 = nn.MaxUnpool2d(2, stride=2)
        self.dec_tconv1 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)

        self.dec_unpool2 = nn.MaxUnpool2d(2, stride=2)
        self.dec_tconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)

        self.dec_unpool3 = nn.MaxUnpool2d(2, stride=2)
        self.dec_tconv3 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)

        self.dec_unpool4 = nn.MaxUnpool2d(2, stride=2)
        self.dec_tconv4 = nn.ConvTranspose2d(64, 3, kernel_size=2, stride=2)

    def forward(self, x):
        """
            Encoder
        """
        x = self.enc_conv1(x)
        x, idx1 = self.enc_pool1(nn.ReLU()(x))

        x = self.enc_conv2(x)
        x, idx2 = self.enc_pool2(nn.ReLU()(x))

        x = self.enc_conv3(x)
        x, idx3 = self.enc_pool3(nn.ReLU()(x))

        x = self.enc_conv4(x)
        x, idx4 = self.enc_pool4(nn.ReLU()(x))
 
        """ 
            Decoder 
        """
        x = self.dec_unpool1(self.dec_tconv1(nn.ReLU()(x)), idx4-1)
        x = self.dec_unpool2(self.dec_tconv2(nn.ReLU()(x)), idx3)
        x = self.dec_unpool3(self.dec_tconv3(nn.ReLU()(x)), idx2)
        x = self.dec_unpool4(self.dec_tconv4(nn.ReLU()(x)), idx1)

        return x


### Training Loop

The learning rate scheduler made a noticeable difference in bringing the training and value losses closer together.

In [16]:
# Initialize the model, loss, and optimizer
ae_model = Autoencoder()
device = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ae_model = ae_model.to(device)

criterion = nn.MSELoss()    
optimizer = optim.SGD(ae_model.parameters(), 
                      lr=0.001, 
                      momentum=0.9)

scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.25, total_iters=10)
# Training loop for 50 epochs
N_EPOCHS = 20
tr_loss_hist = []
val_loss_hist = []
for epoch in range(N_EPOCHS):

    # Training 
    train_loss = 0.0
    ae_model.train() # <1>
    for batch in train_loader:
        images,_ = batch
        images = images.cuda()
        optimizer.zero_grad()

        outputs = ae_model(images)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss += loss.item()

    # Validation
    val_loss = 0.0
    ae_model.eval() # <2>
    for batch in val_loader:
        images,_ = batch
        images = images.cuda()

        outputs = ae_model(images)
        loss = criterion(outputs, images)

        val_loss += loss.item()

    print("Epoch: {} Train Loss: {} Val Loss: {}".format(
                  epoch, 
                  train_loss/len(train_loader), 
                  val_loss/len(val_loader)))
    tr_loss_hist.append(train_loss/len(train_loader))
    val_loss_hist.append(val_loss/len(val_loader))

OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 5.79 GiB total capacity; 3.98 GiB already allocated; 145.25 MiB free; 4.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### Testing

In [ ]:
# Initialize variables to store test losses
total_mse_loss = 0.0
total_mae_loss = 0.0

# Set the model to evaluation mode
ae_model.eval()

# Loop over test data
for x_test_batch, _ in test_loader:  # Note: we don't need labels for autoencoders

    # Move the test data to the same device as the model
    x_test_batch = x_test_batch.to(device)

    # Forward pass to get the model's reconstructed output
    reconstructed_batch = ae_model(x_test_batch)
    print("Shape of x_test_batch:", x_test_batch.shape)
    print("Shape of reconstructed_batch:", reconstructed_batch.shape)

    # Calculate the loss between the reconstructed and original data
    mse_loss = nn.MSELoss()(reconstructed_batch, x_test_batch)
    mae_loss = nn.L1Loss()(reconstructed_batch, x_test_batch)

    # Accumulate the loss
    total_mse_loss += mse_loss.item()
    total_mae_loss += mae_loss.item()

# Calculate the average loss
average_mse_loss = total_mse_loss / len(test_loader)
average_mae_loss = total_mae_loss / len(test_loader)

print(f'Average MSE Loss: {average_mse_loss}')
print(f'Average MAE Loss: {average_mae_loss}')
